In [1]:
from pathlib import Path
from dateutil.parser import parse as parse_date

In [2]:
project_name = 'LA2'
work_dir = Path.cwd() / project_name
data_dir = work_dir / 'data'

stack_start = parse_date('2022-01-01 00:00:00Z')
stack_end = parse_date('2024-03-01 00:00:00Z')
max_temporal_baseline = 27 #days

data_dir.mkdir(parents=True, exist_ok=True)

In [3]:
import asf_search as asf

In [4]:
import pandas as pd

In [5]:
aoi = 'POLYGON((-118.4835 33.6734,-117.8876 33.6734,-117.8876 34.0668,-118.4835 34.0668,-118.4835 33.6734))'
search_results = asf.geo_search(
        platform=asf.PLATFORM.SENTINEL1,
        intersectsWith=aoi,
        start='2022-01-01',
        end='2024-03-01',
        processingLevel=asf.PRODUCT_TYPE.SLC,
        beamSwath='IW',
        flightDirection=asf.FLIGHT_DIRECTION.ASCENDING,
        relativeOrbit=64,
        #asfFrame=109 
    )

/home/jovyan/.local/envs/opensarlab_mintpy_recipe_book/lib/python3.9/site-packages/tzlocal/unix.py:193: UserWarning: Can not find any timezone configuration, defaulting to UTC.
  warnings.warn("Can not find any timezone configuration, defaulting to UTC.")


In [6]:
with open("search_results1.csv", "w") as f:
   f.writelines(search_results.csv())

In [7]:
baseline_results = asf.baseline_search.stack_from_product(search_results[-1])

columns = list(baseline_results[0].properties.keys()) + ['geometry', ]
data = [list(scene.properties.values()) + [scene.geometry, ] for scene in baseline_results]

stack = pd.DataFrame(data, columns=columns)
stack['startTime'] = stack.startTime.apply(parse_date)

stack = stack.loc[(stack_start <= stack.startTime) & (stack.startTime <= stack_end)]

In [8]:
sbas_pairs = set()

for reference, rt in stack.loc[::-1, ['sceneName', 'temporalBaseline']].itertuples(index=False):
    secondaries = stack.loc[
        (stack.sceneName != reference)
        & (stack.temporalBaseline - rt <= max_temporal_baseline)
        & (stack.temporalBaseline - rt > 0)
    ]
    for secondary in secondaries.sceneName:
        sbas_pairs.add((reference, secondary))

In [1]:
import hyp3_sdk as sdk
hyp3 = sdk.HyP3(prompt=True)

NASA Earthdata Login username:  vickielee333
NASA Earthdata Login password:  ········


In [11]:
jobs = sdk.Batch()
for reference, secondary in sbas_pairs:
    jobs += hyp3.submit_insar_job(reference, secondary, name=project_name,
                                  include_dem=True, include_look_vectors=True)

In [4]:
jobs = hyp3.watch(jobs)

NotImplementedError: Cannot watch <class 'str'> type object